In [ ]:
from itertools import product, combinations
import numpy as np
import pandas as pd

# --- Algorithme de Kuo & Zuo pour k-out-of-n:G ---
def kuo_zuo_k_out_of_n(Rs, k):
    n = len(Rs)
    subsystem_reliability = 0.0
    for num_working in range(k, n + 1):
        for working_indices in combinations(range(n), num_working):
            prob = 1.0
            for i in range(n):
                if i in working_indices:
                    prob *= Rs[i]
                else:
                    prob *= (1 - Rs[i])
            subsystem_reliability += prob
    return subsystem_reliability

# --- Configuration du système (extrait de l'article) ---
subsystems = {
    'S1': {'components': ['E11', 'E12', 'E13', 'E14', 'E15'], 'k': 3},
    'S2': {'components': ['E21', 'E22', 'E23', 'E24'], 'k': 2},
    'S3': {'components': ['E31', 'E32', 'E33'], 'k': 2}
}

# Fiabilités des composants (issues du modèle LSTM)
Rij = {
    'E11': 0.893, 'E12': 0.003, 'E13': 0.000, 'E14': 0.003, 'E15': 0.000,
    'E21': 0.860, 'E22': 0.000, 'E23': 0.000, 'E24': 0.000,
    'E31': 0.770, 'E32': 0.000, 'E33': 0.000
}

# Données des niveaux de maintenance
maintenance_levels = {
    0: {'RUL_increase': 0, 'cost': 0.0, 'time': 0.0},
    1: {'RUL_increase': 8, 'cost': 30.0, 'time': 0.8},
    2: {'RUL_increase': 16, 'cost': 75.0, 'time': 2.0},
    3: {'RUL_increase': 32, 'cost': 150.0, 'time': 3.5},
    4: {'RUL_increase': 250, 'cost': 250.0, 'time': 5.0}
}

RUL_max = 250.0  # utilisé pour normaliser les gains de fiabilité

# --- Génération des motifs ---
pattern_data = []

for s_name, s_info in subsystems.items():
    components = s_info['components']
    k = s_info['k']

    for levels in product(range(5), repeat=len(components)):
        total_cost = 0.0
        total_time = 0.0
        updated_reliabilities = []

        for comp, level in zip(components, levels):
            base_r = Rij[comp]
            gain = maintenance_levels[level]['RUL_increase'] / RUL_max
            new_r = min(1.0, base_r + gain)
            updated_reliabilities.append(new_r)
            total_cost += maintenance_levels[level]['cost']
            total_time += maintenance_levels[level]['time']

        R_s_ip = kuo_zuo_k_out_of_n(updated_reliabilities, k)

        pattern_data.append({
            'Subsystem': s_name,
            'Pattern': levels,
            'Cost': total_cost,
            'Time': total_time,
            'Reliability': R_s_ip
        })

# --- Affichage en DataFrame ---
df_patterns = pd.DataFrame(pattern_data)
print(df_patterns.head())  # Pour voir les premières lignes


In [ ]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB

# --- Paramètres globaux ---
n_components = 5
n_scenarios = 100
maintenance_levels = {
    0: {'RUL_increase': 0, 'cost': 0, 'time': 0},
    1: {'RUL_increase': 8, 'cost': 30, 'time': 0.8},
    2: {'RUL_increase': 16, 'cost': 75, 'time': 2},
    3: {'RUL_increase': 32, 'cost': 150, 'time': 3.5},
    4: {'RUL_increase': 250, 'cost': 250, 'time': 5}
}
RUL_max = 250
budget_limit = 600
time_limit = 16

# Stocker les données synthétiques
data = []

for scenario in range(n_scenarios):
    model = Model()
    model.setParam('OutputFlag', 0)

    # Générer des Rij aléatoires pour les composants
    Rij = np.random.uniform(0, 1, n_components)

    z = {}
    for i in range(n_components):
        for l in range(5):
            z[i, l] = model.addVar(vtype=GRB.BINARY, name=f"z_{i}_{l}")
        # Une seule action par composant
        model.addConstr(sum(z[i, l] for l in range(5)) == 1)

    # Contraintes globales de budget et de temps
    model.addConstr(
        sum(z[i, l] * maintenance_levels[l]['cost'] for i in range(n_components) for l in range(5)) <= budget_limit
    )
    model.addConstr(
        sum(z[i, l] * maintenance_levels[l]['time'] for i in range(n_components) for l in range(5)) <= time_limit
    )

    # Objectif : max somme log fiabilité (approximée comme Rij + gain)
    model.setObjective(
        sum(z[i, l] * np.log(min(1.0, Rij[i] + maintenance_levels[l]['RUL_increase'] / RUL_max))
            for i in range(n_components) for l in range(5)),
        GRB.MAXIMIZE
    )

    model.optimize()

    # Extraire les données optimales
    optimal_levels = []
    for i in range(n_components):
        for l in range(5):
            if z[i, l].X > 0.5:
                optimal_levels.append(l)

    for i in range(n_components):
        comp_data = {
            'component_id': i,
            'reliability': Rij[i],
            'optimal_level': optimal_levels[i]
        }
        data.append(comp_data)

# Convertir en DataFrame
df_synthetic = pd.DataFrame(data)
tools.display_dataframe_to_user(name="Synthetic Training Data (From Gurobi)", dataframe=df_synthetic)
